In [1]:
from neuralop.models import FNO 
import sys
import os
sys.path.append(os.path.abspath(".."))
from Models.neural_models import *
from Data import *
from torch.utils.data import DataLoader

Torch Set-Up

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


Create Dataset

In [3]:
# Weather parameters
w = 3
d = 5
n = 1_000

# Select a Probabilistic Model and generate the dataset
prob_model = GaussianData(d, w)
observations = prob_model.generate(n)
dataset = NextDayForecastDataset(observations).to(device)

# Constants
in_channels = dataset.in_channels

Create Probabilistic Neural Network

In [4]:
ACW_FNO = PNN(probabilistic_model = prob_model, 
              model_class = FNO, 
              in_channels = in_channels, 
              hidden_channels = 3, 
              out_channels = 3,
              n_modes = (16, 16),
              grid_shape = dataset.get_input_shape())

# ACW_FNO.use_linear_projection(dataset.get_input_shape(), prob_model.parameter_count)
model = ACW_FNO.to(device)

In [5]:
print(model.project)

Linear(in_features=9, out_features=135, bias=True)


Training Parameters

In [6]:
learning_rate   = 1e-5
batch_size      = 64
EPOCHS          = 10
loss_fn         = NLLLoss(prob_model)
optimizer       = torch.optim.Adam(model.parameters(), lr = learning_rate)

Create DataLoaders

In [7]:
# Train / test split
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.8, 0.2])
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

Define loops

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        # print(f"Batch {batch:>3} : loss = {loss.item():>7f}")

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    num_batches = len(dataloader)
    test_loss = 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

Train the Model

In [9]:
for t in range(EPOCHS):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 16680.964844  [   64/  800]
Test Error: 
 Avg loss: 11558.456787 

Epoch 2
-------------------------------
loss: 13502.725586  [   64/  800]
Test Error: 
 Avg loss: 9416.381470 

Epoch 3
-------------------------------
loss: 10724.991211  [   64/  800]
Test Error: 
 Avg loss: 8334.911499 

Epoch 4
-------------------------------
loss: 8663.058594  [   64/  800]
Test Error: 
 Avg loss: 7797.795776 

Epoch 5
-------------------------------
loss: 8266.564453  [   64/  800]
Test Error: 
 Avg loss: 6591.880737 

Epoch 6
-------------------------------
loss: 7161.727051  [   64/  800]
Test Error: 
 Avg loss: 6201.647095 

Epoch 7
-------------------------------
loss: 6476.707520  [   64/  800]
Test Error: 
 Avg loss: 5174.783936 

Epoch 8
-------------------------------
loss: 6251.520996  [   64/  800]
Test Error: 
 Avg loss: 5500.348145 

Epoch 9
-------------------------------
loss: 4945.094727  [   64/  800]
Test Error: 
 Avg loss: 5032.407104